In [19]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright
import json
import sqlite3


In [20]:
async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        try:
            await page.goto("https://www.sportsbookreview.com/betting-odds/nba-basketball/?date=2021-3-05", wait_until="domcontentloaded")
            locator = page.locator("#__NEXT_DATA__")
            raw_json = await locator.inner_text()
            data = json.loads(raw_json)
        except Exception as e:
            print(f"Error: {e}")
        finally:
            await browser.close()
    return data
data = await main()

In [21]:
dframe = pd.DataFrame(columns=["away_TEAM", "home_TEAM","homeMoneyLine", "awayMoneyLine", "homeSpread","awaySpread"])

In [22]:
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()
dates_20_21 = pd.read_sql_query("SELECT [GAME_DATE] FROM [2020-2021 Team Data Combined]", conn)

In [23]:
dates_20_21 = dates_20_21['GAME_DATE'].unique()
dates_20_21 = dates_20_21.tolist()

In [ ]:
dates_20_21 = [x[0:10] for x in dates_20_21]

['2020-12-22',
 '2020-12-23',
 '2020-12-25',
 '2020-12-26',
 '2020-12-27',
 '2020-12-28',
 '2020-12-29',
 '2020-12-30',
 '2020-12-31',
 '2021-01-01',
 '2021-01-02',
 '2021-01-03',
 '2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-09',
 '2021-01-10',
 '2021-01-11',
 '2021-01-12',
 '2021-01-13',
 '2021-01-14',
 '2021-01-15',
 '2021-01-16',
 '2021-01-17',
 '2021-01-18',
 '2021-01-19',
 '2021-01-20',
 '2021-01-21',
 '2021-01-22',
 '2021-01-23',
 '2021-01-24',
 '2021-01-25',
 '2021-01-26',
 '2021-01-27',
 '2021-01-28',
 '2021-01-29',
 '2021-01-30',
 '2021-01-31',
 '2021-02-01',
 '2021-02-02',
 '2021-02-03',
 '2021-02-04',
 '2021-02-05',
 '2021-02-06',
 '2021-02-07',
 '2021-02-08',
 '2021-02-09',
 '2021-02-10',
 '2021-02-11',
 '2021-02-12',
 '2021-02-13',
 '2021-02-14',
 '2021-02-15',
 '2021-02-16',
 '2021-02-17',
 '2021-02-18',
 '2021-02-19',
 '2021-02-20',
 '2021-02-21',
 '2021-02-22',
 '2021-02-23',
 '2021-02-24',
 '2021-02-25',
 '2021-02-26',
 '2021-02-

In [25]:
for index, x in enumerate((data['props']['pageProps']['oddsTables'][0]['oddsTableModel']['gameRows'])):
    homeMoneyLine, awayMoneyLine, homeSpread, awaySpread = 0,0,0,0
    for z in x["oddsViews"]:
        homeMoneyLine += z['currentLine']["homeOdds"]
        awayMoneyLine += z['currentLine']["awayOdds"]
        homeSpread += z['currentLine']['homeSpread']
        awaySpread += z['currentLine']['awaySpread']
    homeMoneyLine = round(homeMoneyLine/len(x['oddsViews']),2)
    awayMoneyLine = round(awayMoneyLine/len(x['oddsViews']),2)
    homeSpread = round(homeSpread/len(x['oddsViews']),2)
    awaySpread = round(awaySpread/len(x['oddsViews']),2)
    dframe.loc[index] = ({"away_TEAM":x['gameView']['awayTeam']['fullName'],
    "home_TEAM":x['gameView']['homeTeam']['fullName'],
    "homeMoneyLine":homeMoneyLine,
    "awayMoneyLine":awayMoneyLine,
    "homeSpread":homeSpread,
    "awaySpread":awaySpread
    }
    )
dframe

IndexError: list index out of range